In [ ]:
!curl https://ollama.ai/install.sh | sh
!pip install aiohttp pyngrok flask
!pip install ollama ollama-python

In [ ]:
ngrok_auth_token = "<ngrok.com api_key>"


In [27]:
html_file_content = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Ollama Chat Interface</title>
    <script src="https://cdn.tailwindcss.com"></script>
    <link href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.0.0/css/all.min.css" rel="stylesheet">
    <style>
        body {
            font-family: 'Inter', system-ui, -apple-system, BlinkMacSystemFont, sans-serif;
        }
        .message-appear {
            animation: slideIn 0.3s ease-out;
        }
        @keyframes slideIn {
            from {
                opacity: 0;
                transform: translateY(10px);
            }
            to {
                opacity: 1;
                transform: translateY(0);
            }
        }

        /* Modern loading indicator */
        .loading-dots {
            display: flex;
            align-items: center;
            justify-content: center;
            gap: 6px;
            padding: 16px;
        }
        .dot {
            width: 8px;
            height: 8px;
            border-radius: 50%;
            background-color: #6366f1;
            animation: bounce 1.4s infinite ease-in-out both;
        }
        .dot:nth-child(1) { animation-delay: -0.32s; }
        .dot:nth-child(2) { animation-delay: -0.16s; }
        @keyframes bounce {
            0%, 80%, 100% { transform: scale(0); opacity: 0.6; }
            40% { transform: scale(1); opacity: 1; }
        }

        .sidebar-transition {
            transition: all 0.3s ease-in-out;
        }

        /* Markdown styling for chat messages */
        .message-content code {
            # background-color: rgba(99, 102, 241, 0.1);
            padding: 0.2em 0.4em;
            border-radius: 3px;
            font-family: ui-monospace, SFMono-Regular, Menlo, Monaco, Consolas, monospace;
        }


        .message-content .codehilite {
          background: rgba(99, 102, 241, 0.1);
          border-radius: 10px;
        }

        .message-content table {
          border-collapse: separate;
          border-spacing: 0;
          border: 1px solid rgba(99, 102, 241, 0.2);
          border-radius: 6px;
          margin: 10px 0;
          background: rgba(99, 102, 241, 0.1);
          width: 100%;
          overflow: hidden;
        }

        .message-content th {
          border: none;
          padding: 6px 10px;
          text-align: left;
          background: rgba(99, 102, 241, 0.2);
          font-weight: bold;
          border-bottom: 1px solid rgba(99, 102, 241, 0.1);
        }

        .message-content td {
          border: none;
          padding: 6px 10px;
          text-align: left;
          background: rgba(99, 102, 241, 0.1);
        }


        /* Make sure only the outer table has rounded corners */
        .message-content table tr:first-child th:first-child {
          border-top-left-radius: 6px;
        }

        .message-content table tr:first-child th:last-child {
          border-top-right-radius: 6px;
        }

        .message-content table tr:last-child td:first-child {
          border-bottom-left-radius: 6px;
        }

        .message-content table tr:last-child td:last-child {
          border-bottom-right-radius: 6px;
        }

        .message-content tr:nth-child(even) {
          background: rgba(99, 102, 241, 0.1);
        }

        .message-content pre {
            background-color: rgba(30, 41, 59, 0.05);
            padding: 1em;
            border-radius: 8px;
            overflow-x: auto;
            margin: 0.5em 0;
        }
        .message-content p {
            margin-bottom: 0.75em;
        }
        .message-content ul, .message-content ol {
            margin-left: 1.5em;
            margin-bottom: 0.75em;
        }

        /* Custom scrollbar */
        ::-webkit-scrollbar {
            width: 6px;
            height: 6px;
        }
        ::-webkit-scrollbar-track {
            background: #f1f5f9;
            border-radius: 8px;
        }
        ::-webkit-scrollbar-thumb {
            background: #cbd5e1;
            border-radius: 8px;
        }
        ::-webkit-scrollbar-thumb:hover {
            background: #94a3b8;
        }

        /* Modern toggle switch styling */
        .toggle-bg {
            position: relative;
            display: inline-block;
            width: 3.5rem;
            height: 1.75rem;
            background-color: #e2e8f0;
            border-radius: 9999px;
            transition: all 0.2s ease;
        }
        .toggle-bg:before {
            content: '';
            position: absolute;
            top: 0.25rem;
            left: 0.25rem;
            width: 1.25rem;
            height: 1.25rem;
            background-color: white;
            border-radius: 50%;
            box-shadow: 0 1px 3px rgba(0, 0, 0, 0.1);
            transition: transform 0.2s ease;
        }
        input:checked + .toggle-bg {
            background-color: #6366f1;
        }
        input:checked + .toggle-bg:before {
            transform: translateX(1.75rem);
        }

        /* Glass morphism effect for chat bubbles */
        .glass-morphism {
            background: rgba(255, 255, 255, 0.8);
            backdrop-filter: blur(10px);
            -webkit-backdrop-filter: blur(10px);
            border: 1px solid rgba(255, 255, 255, 0.3);
            box-shadow: 0 4px 6px rgba(0, 0, 0, 0.05);
        }
    </style>
</head>
<body class="bg-slate-50 min-h-screen overflow-hidden">
    <div class="flex h-screen">
        <!-- Sidebar -->
        <div id="sidebar" class="sidebar-transition bg-white shadow-md h-full flex flex-col z-10" style="width: 280px;">
            <div class="p-4 border-b border-slate-200 flex items-center justify-between">
                <h2 class="text-xl font-semibold text-slate-800">Chat History</h2>
                <div class="flex items-center space-x-2">
                    <button onclick="startNewChat()" class="bg-indigo-600 text-white p-2 rounded-lg hover:bg-indigo-700 transition-colors duration-200" title="New Chat">
                        <i class="fas fa-plus"></i>
                    </button>
                    <button id="toggleSidebar" class="text-slate-500 p-2 hover:bg-slate-100 rounded-lg" title="Toggle Sidebar">
                        <i class="fas fa-chevron-left"></i>
                    </button>
                </div>
            </div>

            <div class="flex-1 overflow-y-auto p-3">
                <div id="historyList" class="space-y-2">
                    <!-- Chat history will be populated here -->
                </div>
            </div>

            <div class="p-3 bg-slate-50 border-t border-slate-200">
                <button onclick="deleteSelectedSessions()" class="w-full bg-red-500 text-white px-4 py-2 rounded-lg hover:bg-red-600 transition-colors duration-200 flex items-center justify-center gap-2">
                    <i class="fas fa-trash"></i>
                    Delete Selected
                </button>
            </div>
        </div>

        <!-- Collapsed sidebar button -->
        <div id="collapsedSidebar" class="hidden bg-white shadow-md p-3 h-full">
            <button id="expandSidebar" class="text-slate-500 p-2 hover:bg-slate-100 rounded-lg" title="Expand Sidebar">
                <i class="fas fa-chevron-right"></i>
            </button>
        </div>

        <!-- Main content -->
        <div class="flex-1 flex flex-col h-screen bg-gradient-to-br from-slate-50 to-slate-100">
            <!-- Header -->
            <div class="bg-white shadow-sm p-4 flex items-center justify-between">
                <div>
                    <h1 class="text-2xl font-bold text-slate-800">Colab Chat</h1>
                    <p class="text-slate-600 text-sm">Powered by <span id="modelName" class="font-medium text-indigo-600"> </span></p>
                </div>

                <div class="flex items-center space-x-4">
                    <label class="flex items-center space-x-2 cursor-pointer">
                        <span class="text-slate-600 text-sm font-medium">Use Context</span>
                        <div class="relative inline-block">
                            <input type="checkbox" id="useContext" class="sr-only" checked>
                            <div class="toggle-bg"></div>
                        </div>
                    </label>
                </div>
            </div>

            <!-- Chat area -->
            <div id="chatArea" class="flex-1 flex flex-col overflow-hidden">
                <div id="currentChat" class="hidden p-4 mb-2 bg-indigo-50 border border-indigo-100 rounded-lg mx-4 mt-4">
                    <div class="flex items-center justify-between">
                        <div id="currentChatInfo" class="text-indigo-700 text-sm font-medium"></div>
                        <button id="closeCurrentChat" class="text-indigo-500 hover:text-indigo-700">
                            <i class="fas fa-times"></i>
                        </button>
                    </div>
                </div>

                <div id="chatMessages" class="flex-1 overflow-y-auto px-4 py-6 space-y-6">
                    <div class="flex flex-col items-center justify-center h-full text-center text-slate-500">
                        <div class="w-20 h-20 flex items-center justify-center rounded-full bg-indigo-100 mb-6">
                            <i class="fas fa-comments text-4xl text-indigo-500"></i>
                        </div>
                        <h3 class="text-xl font-medium mb-2 text-slate-700">Start a new conversation</h3>
                        <p class="max-w-md text-slate-500">Select a chat from the history or type a message below to begin</p>
                    </div>
                </div>

                <!-- Input area -->
                <div class="p-4 bg-white shadow-inner">
                    <div class="flex gap-3 max-w-4xl mx-auto bg-white rounded-xl shadow-sm p-2">
                        <textarea id="userInput" class="flex-1 p-3 border-none outline-none focus:ring-0 resize-none rounded-lg bg-slate-50" placeholder="Type your message..." rows="1"></textarea>
                        <button onclick="sendMessage()" class="bg-indigo-600 text-white px-5 rounded-lg hover:bg-indigo-700 transition-colors duration-200 flex items-center justify-center">
                            <i class="fas fa-paper-plane"></i>
                        </button>
                    </div>
                </div>
            </div>
        </div>
    </div>

    <script>
        let selectedSessions = new Set();
        let activeChatId = null;
        let chatSessions = [];

        // Initialize the UI
        document.addEventListener('DOMContentLoaded', function() {
            loadChatHistory();

            // Event listeners
            document.getElementById('toggleSidebar').addEventListener('click', toggleSidebar);
            document.getElementById('expandSidebar').addEventListener('click', expandSidebar);
            document.getElementById('closeCurrentChat').addEventListener('click', closeCurrentChat);

            // Handle Enter key
            document.getElementById('userInput').addEventListener('keydown', function(e) {
                if (e.key === 'Enter' && !e.shiftKey) {
                    e.preventDefault();
                    sendMessage();
                }
            });

            // Auto resize textarea
            const textarea = document.getElementById('userInput');
            textarea.addEventListener('input', function() {
                this.style.height = 'auto';
                this.style.height = (this.scrollHeight < 120 ? this.scrollHeight : 120) + 'px';
            });
        });

        async function loadChatHistory() {
            try {
                const response = await fetch('/get_chat_history');
                const data = await response.json();

                if (data.status === 'success') {
                    chatSessions = data.history;
                    renderChatHistory();
                }
            } catch (error) {
                console.error('Error loading chat history:', error);
            }
        }

        function renderChatHistory() {
            const historyList = document.getElementById('historyList');
            historyList.innerHTML = '';

            if (chatSessions.length === 0) {
                historyList.innerHTML = `
                    <div class="text-center p-6 text-slate-500">
                        <div class="flex justify-center">
                            <div class="w-16 h-16 rounded-full bg-slate-100 flex items-center justify-center mb-3">
                                <i class="fas fa-comment-slash text-2xl text-slate-400"></i>
                            </div>
                        </div>
                        <p class="font-medium">No chat history</p>
                    </div>
                `;
                return;
            }

            chatSessions.forEach((session, index) => {
                const firstMessage = session.messages[0] || { user: 'Empty chat', timestamp: new Date().toISOString() };
                const preview = firstMessage.user.length > 30 ? firstMessage.user.substring(0, 30) + '...' : firstMessage.user;
                const timestamp = new Date(firstMessage.timestamp).toLocaleString();

                const chatItem = document.createElement('div');
                chatItem.className = `p-3 rounded-lg shadow-sm transition-all duration-200 cursor-pointer ${session.id === activeChatId ? 'bg-indigo-50 border border-indigo-200' : 'bg-white hover:bg-slate-50 border border-slate-100'}`;
                chatItem.innerHTML = `
                    <div class="flex items-center justify-between">
                        <div class="font-medium ${session.id === activeChatId ? 'text-indigo-700' : 'text-slate-700'}">${session.id}</div>
                        <div class="flex items-center space-x-2">
                            <div class="w-5 h-5 rounded-md border border-slate-200 cursor-pointer chat-select-box flex items-center justify-center transition-colors"
                                 data-index="${index}" onclick="event.stopPropagation()"></div>
                        </div>
                    </div>
                    <div class="${session.id === activeChatId ? 'text-indigo-600' : 'text-slate-600'} mt-1 text-sm">${preview}</div>
                    <div class="text-xs ${session.id === activeChatId ? 'text-indigo-400' : 'text-slate-400'} mt-2">${timestamp}</div>
                `;

                // Add click event to load chat
                chatItem.addEventListener('click', () => loadChat(session.id, index));

                // Add to list
                historyList.appendChild(chatItem);

                // Set selected state if in the selectedSessions set
                const selectBox = chatItem.querySelector('.chat-select-box');
                if (selectedSessions.has(index)) {
                    selectBox.classList.add('bg-indigo-500', 'border-indigo-500');
                    selectBox.innerHTML = '<i class="fas fa-check text-white text-xs"></i>';
                }

                // Add click event for select box
                selectBox.addEventListener('click', function(e) {
                    e.stopPropagation();
                    toggleSessionSelection(this, index);
                });
            });
        }

        function toggleSessionSelection(element, index) {
            if (selectedSessions.has(index)) {
                selectedSessions.delete(index);
                element.classList.remove('bg-indigo-500', 'border-indigo-500');
                element.innerHTML = '';
            } else {
                selectedSessions.add(index);
                element.classList.add('bg-indigo-500', 'border-indigo-500');
                element.innerHTML = '<i class="fas fa-check text-white text-xs"></i>';
            }
        }

        function loadChat(chatId, index) {
            activeChatId = chatId;

            // Update UI to reflect the active chat
            renderChatHistory();

            // Display the chat messages
            const chatMessages = document.getElementById('chatMessages');
            chatMessages.innerHTML = '';

            // Find the chat session
            const session = chatSessions.find(s => s.id === chatId);
            if (!session) return;

            // Show current chat info
            const currentChat = document.getElementById('currentChat');
            const currentChatInfo = document.getElementById('currentChatInfo');
            currentChat.classList.remove('hidden');
            currentChatInfo.textContent = `Viewing chat: ${chatId}`;

            // Add messages
            session.messages.forEach(msg => {
                chatMessages.appendChild(createMessageElement('user', msg.user));
                chatMessages.appendChild(createMessageElement('assistant', msg.assistant));
            });

            // Scroll to bottom
            chatMessages.scrollTop = chatMessages.scrollHeight;
        }

        function closeCurrentChat() {
            activeChatId = null;
            document.getElementById('currentChat').classList.add('hidden');
            document.getElementById('chatMessages').innerHTML = `
                <div class="flex flex-col items-center justify-center h-full text-center text-slate-500">
                    <div class="w-20 h-20 flex items-center justify-center rounded-full bg-indigo-100 mb-6">
                        <i class="fas fa-comments text-4xl text-indigo-500"></i>
                    </div>
                    <h3 class="text-xl font-medium mb-2 text-slate-700">Start a new conversation</h3>
                    <p class="max-w-md text-slate-500">Select a chat from the history or type a message below to begin</p>
                </div>
            `;
            renderChatHistory();
        }

        function createMessageElement(role, content) {
            const messageDiv = document.createElement('div');
            messageDiv.className = 'message-appear flex flex-col gap-2';

            const bubble = document.createElement('div');
            if (role === 'user') {
                bubble.className = 'bg-indigo-600 text-white self-end rounded-2xl rounded-br-none px-4 py-3 max-w-[85%] shadow-sm';
            } else {
                bubble.className = 'glass-morphism text-slate-800 self-start rounded-2xl rounded-tl-none px-4 py-3 max-w-[85%] message-content';
            }

            bubble.innerHTML = content;
            messageDiv.appendChild(bubble);
            return messageDiv;
        }

        async function sendMessage() {
            const userInput = document.getElementById('userInput');
            const chatMessages = document.getElementById('chatMessages');
            const useContext = document.getElementById('useContext').checked;

            if (!userInput.value.trim()) return;

            // Clear welcome message if present
            if (chatMessages.querySelector('.flex.flex-col.items-center.justify-center')) {
                chatMessages.innerHTML = '';
            }

            // Add user message
            chatMessages.appendChild(createMessageElement('user', userInput.value));
            chatMessages.scrollTop = chatMessages.scrollHeight;

            const userMessage = userInput.value;
            userInput.value = '';
            userInput.style.height = 'auto';

            // Show modern loading indicator
            const loadingIndicator = document.createElement('div');
            loadingIndicator.className = 'message-appear flex flex-col gap-2';
            loadingIndicator.innerHTML = `
                <div class="glass-morphism self-start rounded-2xl rounded-tl-none px-4 py-1 max-w-[85%]">
                    <div class="loading-dots">
                        <div class="dot"></div>
                        <div class="dot"></div>
                        <div class="dot"></div>
                    </div>
                </div>
            `;
            chatMessages.appendChild(loadingIndicator);
            chatMessages.scrollTop = chatMessages.scrollHeight;

            try {
                const response = await fetch('/chat', {
                    method: 'POST',
                    headers: {
                        'Content-Type': 'application/json'
                    },
                    body: JSON.stringify({
                        message: userMessage,
                        useContext: useContext,
                        chatId: activeChatId,
                        selectedSessions: Array.from(selectedSessions).map(index => chatSessions[index].id)
                    })
                });

                const data = await response.json();

                // Remove loading indicator
                chatMessages.removeChild(loadingIndicator);

                if (data.status === 'success') {
                    // Update chat history
                    if (data.newChatId && !activeChatId) {
                        activeChatId = data.newChatId;

                        // Show current chat info
                        const currentChat = document.getElementById('currentChat');
                        const currentChatInfo = document.getElementById('currentChatInfo');
                        currentChat.classList.remove('hidden');
                        currentChatInfo.textContent = `Viewing chat: ${activeChatId}`;
                    }

                    // Add assistant message
                    chatMessages.appendChild(createMessageElement('assistant', data.response));
                    chatMessages.scrollTop = chatMessages.scrollHeight;

                    // Reload chat history
                    await loadChatHistory();
                } else {
                    throw new Error(data.message || 'Failed to get response');
                }
            } catch (error) {
                console.error('Error:', error);
                chatMessages.appendChild(createMessageElement('assistant', 'Sorry, there was an error processing your message.'));
            }
        }

        async function deleteSelectedSessions() {
            if (selectedSessions.size === 0) return;

            try {
                const sessionIds = Array.from(selectedSessions).map(index => chatSessions[index].id);

                const response = await fetch('/delete_sessions', {
                    method: 'POST',
                    headers: {
                        'Content-Type': 'application/json'
                    },
                    body: JSON.stringify({
                        sessionIds: sessionIds
                    })
                });

                const data = await response.json();

                if (data.status === 'success') {
                    // Clear selections
                    selectedSessions.clear();

                    // If active chat was deleted, close it
                    if (sessionIds.includes(activeChatId)) {
                        closeCurrentChat();
                    }

                    // Reload chat history
                    await loadChatHistory();
                } else {
                    throw new Error(data.message || 'Failed to delete sessions');
                }
            } catch (error) {
                console.error('Error:', error);
                alert('Failed to delete sessions');
            }
        }

        async function startNewChat() {
            // Clear current chat if any
            closeCurrentChat();

            // Clear selected sessions
            selectedSessions.clear();

            try {
                const response = await fetch('/clear_current_session', {
                    method: 'POST',
                    headers: {
                        'Content-Type': 'application/json'
                    }
                });

                const data = await response.json();

                if (data.status === 'success') {
                    document.getElementById('userInput').focus();
                } else {
                    throw new Error(data.message || 'Failed to start new chat');
                }
            } catch (error) {
                console.error('Error:', error);
            }
        }

        // Toggle sidebar functions
        function toggleSidebar() {
            const sidebar = document.getElementById('sidebar');
            const collapsedSidebar = document.getElementById('collapsedSidebar');

            sidebar.style.width = '0';
            setTimeout(() => {
                sidebar.classList.add('hidden');
                collapsedSidebar.classList.remove('hidden');
            }, 300);
        }

        function expandSidebar() {
            const sidebar = document.getElementById('sidebar');
            const collapsedSidebar = document.getElementById('collapsedSidebar');

            collapsedSidebar.classList.add('hidden');
            sidebar.classList.remove('hidden');
            setTimeout(() => {
                sidebar.style.width = '280px';
            }, 10);
        }
    </script>
</body>
</html>
"""

In [28]:
import os
if not os.path.exists('templates'):
    os.makedirs('templates')
with open('templates/index.html', 'w') as f:
    f.write(html_file_content)

In [29]:
import os
import time
import markdown
import subprocess
import json
import uuid
from flask import Flask, render_template, request, jsonify, send_from_directory
from datetime import datetime
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import ollama
from pyngrok import ngrok

# Start Ollama server
ollama_model = "llama3.2"
os.environ.update({'OLLAMA_HOST': '0.0.0.0', 'LD_LIBRARY_PATH': '/usr/lib64-nvidia'})
ollama_process = subprocess.Popen(["ollama", "serve"])
time.sleep(5)
subprocess.Popen(["ollama", "pull", ollama_model])

app = Flask(__name__)

CHAT_HISTORY_FILE = 'chat_history.json'

# Load and save chat history functions
def load_chat_history():
    if os.path.exists(CHAT_HISTORY_FILE):
        with open(CHAT_HISTORY_FILE, 'r') as file:
            return json.load(file)
    return []

def save_chat_history(history):
    with open(CHAT_HISTORY_FILE, 'w') as file:
        json.dump(history, file, indent=2)



# Context retrieval function
def get_relevant_context(history, current_input, max_context=3):
    if not history:
        return []

    previous_messages = [f"{msg['user']} {msg['assistant']}" for session in history for msg in session['messages']]
    all_messages = previous_messages + [current_input]

    vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = vectorizer.fit_transform(all_messages)

    current_vector = tfidf_matrix[-1:]
    similarities = cosine_similarity(current_vector, tfidf_matrix[:-1])[0]

    # Get the most relevant message indices
    message_count = len(previous_messages)
    most_relevant_indices = np.argsort(similarities)[-max_context:]

    # Map these indices back to the original session and message
    relevant_messages = []
    message_index = 0
    for session in history:
        for msg in session['messages']:
            if message_index in most_relevant_indices:
                relevant_messages.append(msg)
            message_index += 1
            if message_index >= message_count:
                break
        if message_index >= message_count:
            break

    return relevant_messages

@app.route('/')
def index():
    try:
        return render_template('index.html')
    except Exception as e:
        print(f"Error in index(): {str(e)}")
        return f"An error occurred: {str(e)}", 500

@app.route('/templates/<path:path>')
def serve_template(path):
    return send_from_directory('templates', path)

@app.route('/get_chat_history')
def get_chat_history():
    try:
        history = load_chat_history()
        return jsonify({
            'status': 'success',
            'history': history
        })
    except Exception as e:
        print(f"Error in get_chat_history(): {str(e)}")
        return jsonify({
            'status': 'error',
            'message': str(e)
        }), 500


@app.route('/chat', methods=['POST'])
def chat_endpoint():
    try:
        data = request.json
        user_input = data.get('message', '')
        use_context = data.get('useContext', True)
        chat_id = data.get('chatId')
        selected_sessions = data.get('selectedSessions', [])

        history = load_chat_history()

        messages = [{"role": "system", "content": "You are a helpful AI assistant."}]

        if use_context:
            context = []
            if selected_sessions:
                for session in history:
                    if session['id'] in selected_sessions:
                        for msg in session['messages']:
                            context.append(msg)
            else:
                context = get_relevant_context(history, user_input)

            for entry in context:
                messages.append({"role": "user", "content": entry['user']})
                messages.append({"role": "assistant", "content": entry['assistant']})

        messages.append({"role": "user", "content": user_input})

        response = ollama.chat(model=ollama_model, messages=messages)
        assistant_response = response['message']['content'].strip()

        # Format response using Markdown
        # formatted_response = markdown.markdown(assistant_response)
        formatted_response = markdown.markdown(assistant_response, extensions=['fenced_code', 'codehilite', 'tables'])

        new_message = {
            'user': user_input,
            'assistant': formatted_response,
            'timestamp': datetime.now().isoformat()
        }

        new_chat_id = None
        if not chat_id:
            new_chat_id = f"Chat-{uuid.uuid4().hex[:8]}"
            history.append({'id': new_chat_id, 'messages': [new_message]})
        else:
            for session in history:
                if session['id'] == chat_id:
                    session['messages'].append(new_message)
                    break

        save_chat_history(history)

        return jsonify({
            'status': 'success',
            'response': formatted_response,
            'newChatId': new_chat_id
        })

    except Exception as e:
        print(f"Error in chat_endpoint: {str(e)}")
        return jsonify({'status': 'error', 'message': str(e)}), 500


@app.route('/delete_sessions', methods=['POST'])
def delete_sessions():
    try:
        session_ids = request.json.get('sessionIds', [])
        history = load_chat_history()

        # Filter out sessions to be deleted
        history = [session for session in history if session['id'] not in session_ids]

        save_chat_history(history)
        return jsonify({'status': 'success', 'history': history})

    except Exception as e:
        return jsonify({
            'status': 'error',
            'message': str(e)
        }), 500

@app.route('/clear_current_session', methods=['POST'])
def clear_current_session():
    try:
        return jsonify({'status': 'success'})
    except Exception as e:
        return jsonify({'status': 'error', 'message': str(e)}), 500

# Start server with ngrok
def run_with_ngrok(ngrok_auth_token):
    # Start ngrok
    ngrok.set_auth_token(ngrok_auth_token)
    public_url = ngrok.connect(5000).public_url
    print(f"* Ngrok tunnel active at {public_url}")
    print(f"* Open this URL in your browser to access the app")

    # Run Flask app
    app.run(host='0.0.0.0', port=5000, debug=True, use_reloader=False)

if __name__ == '__main__':
    if ngrok_auth_token:
        run_with_ngrok(ngrok_auth_token)
    else:
        app.run(host='0.0.0.0', port=5000, debug=True)

* Ngrok tunnel active at https://fd0e-34-168-145-209.ngrok-free.app
* Open this URL in your browser to access the app
 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [26/Feb/2025 06:44:00] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Feb/2025 06:44:01] "GET /get_chat_history HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Feb/2025 06:44:01] "GET /favicon.ico HTTP/1.1" 404 -


In [30]:
ngrok.kill()

In [31]:
def cleanup():
    print("Shutting down Ollama server...")
    ollama_process.terminate()
